In [34]:
import pandas as pd
import numpy as np

# Read Data1.xlsx into a dataframe
orders_data_fy17_fy21 = pd.read_excel('data_files/Data1.xlsx')

# Read Data2.xlsx into a dataframe
orders_data_fy21_fy22 = pd.read_excel('data_files/Data2.xlsx')

In [35]:
# Keep desired columns
temp_df = orders_data_fy17_fy21[['year_month', 'company_code_n', 'sold_to_customer_n_latest', 'bu', 'bu_n', 'material', 'ms_code', 'order_intake_quantity', 'order intake EUR']]

# Parse 'year_month' column and create 'FY' column. FY start from 4 month
temp_df['year_month_date'] = pd.to_datetime(temp_df['year_month'], format='%Y%m')
temp_df['FY'] = np.where(temp_df['year_month_date'].dt.month >= 4, temp_df['year_month_date'].dt.year, temp_df['year_month_date'].dt.year - 1)

# Create fiscal 'quarter' column
quarter_dict = {1: 'Q4', 2: 'Q4', 3: 'Q4', 4: 'Q1', 5: 'Q1', 6: 'Q1', 7: 'Q2', 8: 'Q2', 9: 'Q2', 10: 'Q3', 11: 'Q3', 12: 'Q3'}
temp_df['quarter'] = temp_df['year_month_date'].dt.month.map(quarter_dict)

# Create 'half_year' column
temp_df['half_year'] = np.where(temp_df['year_month_date'].dt.month.between(4, 9), 'HY1', 'HY2')

# Rename columns
temp_df = temp_df.rename(columns={'sold_to_customer_n_latest': 'customer', 'order intake EUR': 'order_intake_EUR'})


In [25]:
# Check new_df
new_df.to_excel('data_files/new_file.xlsx', index=False)

In [ ]:
temp_df2 = orders_data_fy21_fy22[['year_month', 'company_code_n', 'sold_to_customer_n_latest', 'bu', 'bu_n', 'material', 'ms_code', 'order_intake_quantity', 'Order Intake Euro']]

# Parse 'year_month' column and create 'FY' column. FY start from 4 month
temp_df2['year_month_date'] = pd.to_datetime(temp_df2['year_month'], format='%Y%m')
temp_df2['FY'] = np.where(temp_df2['year_month_date'].dt.month >= 4, temp_df2['year_month_date'].dt.year, temp_df2['year_month_date'].dt.year - 1)

# Create fiscal 'quarter' column
quarter_dict = {1: 'Q4', 2: 'Q4', 3: 'Q4', 4: 'Q1', 5: 'Q1', 6: 'Q1', 7: 'Q2', 8: 'Q2', 9: 'Q2', 10: 'Q3', 11: 'Q3', 12: 'Q3'}
temp_df2['quarter'] = temp_df2['year_month_date'].dt.month.map(quarter_dict)

# Create 'half_year' column
temp_df2['half_year'] = np.where(temp_df2['year_month_date'].dt.month.between(4, 9), 'HY1', 'HY2')

# Rename columns
temp_df2 = temp_df2.rename(columns={'sold_to_customer_n_latest': 'customer', 'Order Intake Euro': 'order_intake_EUR'})

In [32]:
# Check new_df2
temp_df2.to_excel('data_files/new_file2.xlsx', index=False)

In [37]:
# combine two prepared df
orders_data = pd.concat([temp_df, temp_df2], ignore_index=True)
# Check sales_data
orders_data.to_excel('data_files/orders_data.xlsx', index=False)